# Robustness quantification in continuous-time data-driven estimation


Ayush Pandey

March 29, 2025

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LinearRegression
import control as ct
import sympy as sp
import plotly.graph_objects as go
import plotly.express as px

### Linear Parameter Varying (LPV) Systems
Consider a mass-spring damper system with a scheduling parameter $\theta_1$. The states ($x$) of a simplified 2D dynamics are the position ($q$) and velocity ($\dot{q}$) of the mass object. The dynamics are given by:  

$ \dot{x} = A(\theta)x + Bu $

$ y = Cx + Du $

where 

$ A(\theta) = \begin{bmatrix} 0 & 1 \\ -(20 + 5\theta_1) & -(2 + 0.5\theta_1) \end{bmatrix} $

$ B = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, \quad C = \begin{bmatrix} 1 & 0 \end{bmatrix}, \quad D = 0 $

The set of parameters $\theta$ is $\left\{ \theta_1 \right\}$, with just one parameter.

We can write $A(\theta)$ in an affine form as:

$ A(\theta) = A_0 + A_1\theta$

with

$ A_0 = \begin{bmatrix} 0 & 1 \\ -20 & -2 \end{bmatrix} , \quad  A_1 = \begin{bmatrix} 0 & 0 \\ -5 & -0.5 \end{bmatrix} $.


An estimated system dynamics is given by

$\tilde{A} = \begin{bmatrix}0 & 1 \\ -(19.80 + 5.10\theta_1) & -(2.05 + 0.48\theta_1)\end{bmatrix}$

with a clear affine form.

Now, to apply Theorem 1, we have

$\bar{A} = \begin{bmatrix} 0 & 1 & 0 & 0\\ -(20 + 5\theta_1) & -(2 + 0.5\theta_1) & 0 & 0 \\ 0 & 0 & 0 & 1 \\0 & 0 &  -(19.80 + 5.10\theta_1) & -(2.05 + 0.48\theta_1)\end{bmatrix}$.

Then, the derivative of $\bar{A}$ with respect to $\theta_1$ is given by (computed using sympy)

$\bar{A} = \begin{bmatrix} 0 & 0 & 0 & 0\\ -5 & -0.5 & 0 & 0 \\ 0 & 0 & 0 & 0 \\0 & 0 &  -5.10 & -0.48\end{bmatrix}$.


Write the system dynamics

In [2]:
from main import System, AugmentedSystem
import pandas as pd
import sympy as sp

# Define symbolic LPV model
theta1 = sp.symbols('theta1', real=True)
param_syms = [theta1]

# True system symbolic matrices
A0 = sp.Matrix([[0, 1],
                [-20, -2]])
A1 = sp.Matrix([[0, 0],
                [-5, -sp.Rational(1,2)]])  # -0.5
A_theta = A0 + A1*theta1
B = sp.Matrix([[0],[1]])
C = sp.Matrix([[1,0]])
D = sp.Matrix([[0]])

sys_true = System(A_theta, B, C, D, param_syms)

# Estimated system symbolic matrices
A0_t = sp.Matrix([[0, 1],
                  [-19.80, -2.05]])
A1_t = sp.Matrix([[0, 0],
                  [-5.10, -0.48]])
Atilde_theta = A0_t + A1_t*theta1
sys_tilde = System(Atilde_theta, B, C, D, param_syms)

# Augmented system
aug_sys = AugmentedSystem(sys_true, sys_tilde)


Matrix([[0, 0], [0, 0]])


ValueError: B and D must have the same number of columns.

In [4]:
D

Matrix([[0]])

In [4]:
A_theta

Matrix([
[             0,             1],
[-5*theta1 - 20, -theta1/2 - 2]])

Write matrices using sympy and then differentiate symbolically

In [ ]:
# Augmented/stacked system symbolic matrices
Abar_sym = aug_sys.A_sym
Bbar_sym = aug_sys.B_sym
Cbar_sym = aug_sys.C_sym
Dbar_sym = aug_sys.D_sym

# Derivative w.r.t. theta_1
# For generality, differentiate w.r.t. all parameters
if aug_sys.param_syms:
    dAbar_dtheta_sym = sp.diff(Abar_sym, aug_sys.param_syms[0])
else:
    dAbar_dtheta_sym = None


In [ ]:
dAbar_dtheta_sym  # Symbolic derivative from AugmentedSystem class


Matrix([
[ 0,    0,    0,     0],
[-5, -1/2,    0,     0],
[ 0,    0,    0,     0],
[ 0,    0, -0.1, -4.48]])

Lambdify the matrices for numerical computation

In [ ]:
# Lambdify using class methods
f_Abar = aug_sys.f_A
f_dAbar = sp.lambdify(aug_sys.param_syms, dAbar_dtheta_sym, 'numpy')

# Experiment setup
T_final = 1.0
dt = 0.01
Tvec = np.arange(0.0, T_final+dt, dt)

# assume sinusoidal input (any other input is fine too!)
u_amp = 0.1
u_freq = 20
Uvec = u_amp * np.sin(2*np.pi*u_freq*Tvec)

x0_true = np.array([0.2, 0.0])
x0_est  = np.array([0.2, 0.0])
xbar0 = np.concatenate([x0_true, x0_est])
theta_grid = np.linspace(0.0, 1, 6)  
delta_theta = 1e-2
data = []

   theta  ybar_inf  d_ybar_dtheta_inf  bound_rhs        K1        K2  \
0    0.0  0.000089       1.177156e-08   0.399004  0.000064  0.000025   
1    0.2  0.000065       5.794908e-09   0.378752  0.000055  0.000019   
2    0.4  0.000053       2.869805e-09   0.360468  0.000047  0.000015   
3    0.6  0.000051       1.426709e-09   0.343878  0.000041  0.000012   
4    0.8  0.000053       7.148031e-10   0.328755  0.000036  0.000009   
5    1.0  0.000057       3.639746e-10   0.314912  0.000032  0.000008   

         K3         mu   dA_norm    Bu_inf  
0  0.165357   8.552487  5.024938  0.095106  
1  0.157048   9.004974  5.024938  0.095106  
2  0.149534   9.457462  5.024938  0.095106  
3  0.142706   9.909950  5.024938  0.095106  
4  0.136475  10.362439  5.024938  0.095106  
5  0.130765  10.814929  5.024938  0.095106  
Symbolic A_bar(theta):
⎡    0          1            0                0       ⎤
⎢                                                     ⎥
⎢              θ₁                            

In [69]:
import plotly.graph_objects as go
fig = go.Figure()
# plot log y-axis

fig.add_trace(go.Scatter(
    x=df['theta'], y=df['d_ybar_dtheta_inf'],
    mode='lines+markers',
    marker=dict(symbol='circle'),
    name='$\| \\partial \\bar{y}/ \\partial \\theta \|_{\\infty}$ (empirical)',
))

fig.add_trace(go.Scatter(
    x=df['theta'], y=df['bound_rhs'],
    mode='lines+markers',
    marker=dict(symbol='square'),
    name='Theorem 1 bound'
))

fig.update_layout(
    xaxis_title='theta',
    yaxis_title='Sensitivity of estimation error',
    title='Bound vs. empirical norms across theta',
    legend=dict(font=dict(size=12)),
    width=800,
    height=500,
    yaxis_type='log'
)
fig.show()